In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q,
    boost_dict = boost,
    num_results = 5
)

In [10]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [11]:
filtered_results = index.search(
    query = q,
    filter_dict = {'course': 'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results = 5
)

In [12]:
filtered_results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [13]:
from openai import OpenAI

In [14]:
client = OpenAI()

In [15]:
response = client.chat.completions.create(
                model = 'gpt-4o',
                messages = [{"role": "user", "content":q}]
                 )

In [16]:
response

ChatCompletion(id='chatcmpl-BgY5IPmkOsGLaCOmXpspTRahILTgS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="If the course has already started, enrollment policies can vary depending on the institution or organization offering the course. Here are a few general steps you can take to find out if you can still enroll:\n\n1. **Check the Course Website:** Look for any information related to late enrollment or add/drop deadlines. Some courses may allow late registration within a certain time frame.\n\n2. **Contact the Instructor or Course Coordinator:** Reach out via email or through the course's contact information to inquire about the possibility of joining late. Be sure to mention your interest and any relevant background that might support your case.\n\n3. **Contact the Admissions Office or Registrar:** If the course is part of a larger institution, the admissions office or registrar might have policies regarding late enrollment. They 

In [17]:
response.choices[0].message.content

"If the course has already started, enrollment policies can vary depending on the institution or organization offering the course. Here are a few general steps you can take to find out if you can still enroll:\n\n1. **Check the Course Website:** Look for any information related to late enrollment or add/drop deadlines. Some courses may allow late registration within a certain time frame.\n\n2. **Contact the Instructor or Course Coordinator:** Reach out via email or through the course's contact information to inquire about the possibility of joining late. Be sure to mention your interest and any relevant background that might support your case.\n\n3. **Contact the Admissions Office or Registrar:** If the course is part of a larger institution, the admissions office or registrar might have policies regarding late enrollment. They can provide guidance on what steps to take.\n\n4. **Consider Online Courses:** If the course in question is inflexible with late enrollment, you might find simi

In [18]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [19]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [20]:
print(context)

section: General course-related questions
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you c

In [21]:
prompt = prompt_template.format(question=q, context = context).strip()

In [22]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, t

In [23]:
response = client.chat.completions.create(
                model = 'gpt-4o',
                messages = [{"role": "user", "content":prompt}]
                 )

response.choices[0].message.content

'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course. In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.'

### 1.5 The RAG Flow  - Cleaning and Modularizing Code

In [24]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )

    return results

In [25]:
def build_prompt(query, search_results):
    prompt_template = """
                        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
                        Use only the facts from the CONTEXT when answering the QUESTION.
                        
                        QUESTION: {question}
                        
                        CONTEXT: 
                        {context}
                        """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [26]:
def llm(prompt):
    response = client.chat.completions.create(
                model = 'gpt-4o',
                messages = [{"role": "user", "content":prompt}]
                 )

    return response.choices[0].message.content

In [27]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [28]:
rag(query)

'To run Kafka in a Java environment, navigate to the project directory and execute the following command in the terminal:\n\n```shell\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nIf you are working with Python and encountering issues with the module not being found, create and activate a virtual environment, and install the necessary dependencies as specified in the `requirements.txt` file before running your Python Kafka files.'

In [29]:
rag('the course has already started, can I still enroll?')

'Yes, you can still enroll even after the course has started. You are eligible to submit the homework, but keep in mind that there will be deadlines for submitting the final projects, so make sure not to leave everything until the last minute.'

## Search with Elasticsearch

Elastic search is persistent. That is, it saves all data on disk and next time when you start. You do not have to start from scratch.

In [30]:
from elasticsearch import Elasticsearch

In [33]:
es_client = Elasticsearch('http://127.0.0.1:9200')

In [34]:
es_client.info()

ObjectApiResponse({'name': '3328178cc9ce', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'wyML9guZQwu_Yqn8R2q-9g', 'version': {'number': '9.0.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '73f7594ea00db50aa7e941e151a5b3985f01e364', 'build_date': '2025-04-30T10:07:41.393025990Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [35]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

if not es_client.indices.exists(index=index_name):
    es_client.indices.create(index=index_name, body=index_settings)
else:
    print(f"Index '{index_name}' already exists.")


In [36]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
for doc in tqdm(documents):
    es_client.index(index = index_name, document = doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:04<00:00, 228.79it/s]


In [38]:
query = 'I just discovered the course. Can I still join?'

In [39]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], #question is 3 times more important than text and section
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index = index_name, body = search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [40]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [41]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [42]:
rag(query)

'Yes, you can still join the course even if you discover it after the start date. You are eligible to submit the homework, but keep in mind there will be deadlines for turning in the final projects. So, avoid leaving everything for the last minute.'